# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging

from matplotlib import pyplot as plt
import pandas as pd
import os

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.automl.core.featurization import FeaturizationConfig
from azureml.core.dataset import Dataset
from azureml.train.automl import AutoMLConfig


from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails

In [2]:
ws = Workspace.from_config()
experiment_name = 'Custom_Health_Insurance_CS'

exp = Experiment(ws, experiment_name)


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
found = False
key = "Health insurance cross sell"

#dataset = Dataset.get_by_name(workspace, name='Health insurance cross sell')
#dataset.to_pandas_dataframe()

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        dl_data = 'https://www.kaggle.com/anmolkumar/health-insurance-cross-sell-prediction/download'
        dataset = Dataset.Tabular.from_delimited_files(dl_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description="https://www.kaggle.com/anmolkumar/health-insurance-cross-sell-prediction")

data = dataset.to_pandas_dataframe()

In [4]:
amlcompute_cluster_name = "MS-CPUcompute"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)
compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 2)

Found existing cluster, use it.
Succeeded........................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [5]:
automl_config = AutoMLConfig(
    experiment_timeout_minutes=90,
    compute_target = compute_target,
    training_data = dataset,
    task='classification',
    primary_metric='accuracy',
    label_column_name='Response',
    n_cross_validations=5,
    enable_early_stopping = True,
    max_concurrent_iterations = 4,
    max_cores_per_iteration = -1,
    featurization = 'auto',
    verbosity = logging.INFO
)


In [6]:
# TODO: Submit your experiment
remote_run = exp.submit(automl_config)
RunDetails(remote_run).show()

Running on remote.


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [7]:
#remote_run = exp.submit(automl_config, show_output = False)

In [8]:
remote_run.wait_for_completion()

{'runId': 'AutoML_78ecd25e-4e01-4e75-916d-03293f5b0239',
 'target': 'MS-CPUcompute',
 'status': 'Completed',
 'startTimeUtc': '2021-02-03T15:14:02.278029Z',
 'endTimeUtc': '2021-02-03T16:21:16.511629Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'MS-CPUcompute',
  'AMLSettingsJsonString': '{"path":null,"name":"Custom_Health_Insurance_CS","subscription_id":"336509a3-005c-4650-b1e5-e1c99e57a5e1","resource_group":"ms_test","workspace_name":"michaels_test1","region":"northeurope","compute_target":"MS-CPUcompute","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":null,"y_max":null,"num_classes":null,"featuriz

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [9]:
# Wait for the best model explanation run to complete
from azureml.core.run import Run
model_explainability_run_id = remote_run.id + "_" + "ModelExplain"
print(model_explainability_run_id)
model_explainability_run = Run(experiment=exp, run_id=model_explainability_run_id)
model_explainability_run.wait_for_completion()

AutoML_78ecd25e-4e01-4e75-916d-03293f5b0239_ModelExplain


{'runId': 'AutoML_78ecd25e-4e01-4e75-916d-03293f5b0239_ModelExplain',
 'target': 'MS-CPUcompute',
 'status': 'Completed',
 'startTimeUtc': '2021-02-03T16:21:28.54075Z',
 'endTimeUtc': '2021-02-03T16:26:40.755456Z',
 'properties': {'azureml.runsource': 'automl',
  'parentRunId': 'AutoML_78ecd25e-4e01-4e75-916d-03293f5b0239_39',
  '_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': '8aaca651-1e6f-41a2-9674-77bcc18d8143',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json',
  'dependencies_versions': '{"azureml-train-automl-runtime": "1.21.0", "azureml-train-automl-client": "1.21.0", "azureml-telemetry": "1.21.0", "azureml-pipeline-core": "1.21.0", "azureml-model-management-sdk": "1.0.1b6.post1", "azureml-interpret": "1.21.0", "azureml-defaults": "1.21.0", "azureml-dataset-runtime": "1.21.0", "azureml-dataprep": "2.8.2", "azureml-dataprep-rslex": "1.6.0", "azureml-dataprep-native": "28.0.0", "azureml-core": "1.21.

In [10]:
best_automl_run, fitted_model = remote_run.get_output()
best_automl_run

Package:azureml-automl-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-core, training version:1.21.0.post1, current version:1.20.0
Package:azureml-dataprep, training version:2.8.2, current version:2.7.3
Package:azureml-dataprep-native, training version:28.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.6.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-defaults, training version:1.21.0, current version:1.20.0
Package:azureml-interpret, training version:1.21.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.21.0, current version:1.20.0
Package:azureml-telemetry, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.21.0, current version:1.20.0


Experiment,Id,Type,Status,Details Page,Docs Page
Custom_Health_Insurance_CS,AutoML_78ecd25e-4e01-4e75-916d-03293f5b0239_39,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [11]:
best_automl_run.get_tags()

{'_aml_system_azureml.automlComponent': 'AutoML',
 '_aml_system_ComputeTargetStatus': '{"AllocationState":"steady","PreparingNodeCount":0,"RunningNodeCount":2,"CurrentNodeCount":2}',
 'ensembled_iterations': '[22, 23, 18, 7, 35, 4, 29, 1]',
 'ensembled_algorithms': "['XGBoostClassifier', 'XGBoostClassifier', 'XGBoostClassifier', 'XGBoostClassifier', 'XGBoostClassifier', 'RandomForest', 'XGBoostClassifier', 'XGBoostClassifier']",
 'ensemble_weights': '[0.2857142857142857, 0.07142857142857142, 0.07142857142857142, 0.14285714285714285, 0.07142857142857142, 0.14285714285714285, 0.14285714285714285, 0.07142857142857142]',
 'best_individual_pipeline_score': '0.8775625875048852',
 'best_individual_iteration': '22',
 '_aml_system_automl_is_child_run_end_telemetry_event_logged': 'True',
 'model_explain_run_id': 'AutoML_78ecd25e-4e01-4e75-916d-03293f5b0239_ModelExplain',
 'model_explanation': 'True'}

In [12]:
best_automl_run.register_model(model_name = "automl.pkl", model_path = './outputs/')
print(fitted_model._final_estimator)

PreFittedSoftVotingClassifier(classification_labels=None,
                              estimators=[('22',
                                           Pipeline(memory=None,
                                                    steps=[('standardscalerwrapper',
                                                            <azureml.automl.runtime.shared.model_wrappers.StandardScalerWrapper object at 0x7f6bc3f735c0>),
                                                           ('xgboostclassifier',
                                                            XGBoostClassifier(base_score=0.5,
                                                                              booster='gbtree',
                                                                              colsample_bylevel=1,
                                                                              colsample_bynode=1,
                                                                              colsample_bytree=0.8...
                 

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service